In [2]:
## Load packages ----
import numpy as np
import pandas as pd
import sshtunnel
import psycopg2 as pg
import os
import seaborn as sns
import matplotlib.pyplot as plt
import json

/n/home_fasse/laurenflynn/.conda/envs/dorieh/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [22]:
## Open ssh tunnel to DB host ----
tunnel = sshtunnel.SSHTunnelForwarder(
    ('nsaph.rc.fas.harvard.edu', 22),
    ssh_username=f'{os.environ["MY_NSAPH_SSH_USERNAME"]}',
    ssh_private_key=f'{os.environ["HOME"]}/.ssh/id_rsa', 
    ssh_password=f'{os.environ["MY_NSAPH_SSH_PASSWORD"]}', 
    remote_bind_address=("localhost", 5432)
)

tunnel.start()

In [23]:
## Open connection to DB ----
connection = pg.connect(
    host='localhost',
    database='nsaph2',
    user=f'{os.environ["MY_NSAPH_DB_USERNAME"]}',
    password=f'{os.environ["MY_NSAPH_DB_PASSWORD"]}', 
    port=tunnel.local_bind_port
)

In [25]:
def get_outcomes():
    """ Get and return ICD codes """""
    f = open('icd_codes.json')
    outcomes_ = json.load(f)
    f.close()
    return json.loads(outcomes_[0])

def get_counts_query(diagnoses):
    """
    calculates the number of mental health hospitalizations per county per year
    """
    sql_query = f"""
    WITH d AS ( 
        SELECT DISTINCT
            year, 
            bene_id
        FROM 
            medicaid.admissions
        WHERE 
            diagnosis::text[] && ARRAY{diagnoses}
        ), 
        y AS (
            SELECT DISTINCT residence_county, year 
            FROM medicaid.enrollments 
        )
        SELECT 
            y.residence_county , y.year,
            COALESCE(COUNT(d.bene_id), 0) AS n
        FROM y 
        LEFT JOIN d
        ON y.year = d.year
        GROUP BY y.year, y.residence_county
        """
    
    
    return sql_query

In [ ]:
diags = get_outcomes()
mentalHealthDiagnosesICD9 = diags['mental_health']['icd9']
mentalHealthDiagnosesICD10 = diags['mental_health']['icd10']
mentalHealthDiagnoses = mentalHealthDiagnosesICD9 + mentalHealthDiagnosesICD10

In [ ]:
#currently have medicaid data through 2012 so we will only use icd9

sql_query= get_counts_query(mentalHealthDiagnosesICD9)
data = pd.read_sql_query(sql_query, connection, index_col = 'bene_id').reset_index()

In [5]:
adjustmentReactionICD9 = diags['adjustment_reaction']['icd9']
adjustmentReactionICD10 = diags['adjustment_reaction']['icd10']
adjustmentReaction = adjustmentReactionICD9 + adjustmentReactionICD10

In [26]:
#currently have medicaid data through 2012 so we will only use icd9

sql_query= get_counts_query(adjustmentReactionICD9)
print(sql_query)


    WITH d AS ( 
        SELECT DISTINCT
            year, 
            bene_id
        FROM 
            medicaid.admissions
        WHERE 
            diagnosis::text[] && ARRAY['3090', '3091', '30922', '30923', '30924', '30928', '30929', '3093', '3094', '30982', '30983', '30989', '3099']
        ), 
        y AS (
            SELECT DISTINCT residence_county, year 
            FROM medicaid.enrollments 
        )
        SELECT 
            y.residence_county , y.year,
            COALESCE(COUNT(d.bene_id), 0) AS n
        FROM y 
        LEFT JOIN d
        ON y.year = d.year
        GROUP BY y.year, y.residence_county
        


In [27]:
data = pd.read_sql_query(sql_query, connection, index_col = 'bene_id').reset_index()

KeyError: "None of ['bene_id'] are in the columns"

In [ ]:
print(data.head())

In [6]:
anxietyDisordersICD9 = diags['anxiety_disorders']['icd9']
anxietyDisordersICD10 = diags['anxiety_disorders']['icd10']
anxietyDisorders = anxietyDisordersICD9 + anxietyDisordersICD10

In [7]:
attentionDisordersICD9 = diags['attention_disorders']['icd9']
attentionDisordersICD10 = diags['attention_disorders']['icd10']
attentionDisorders = attentionDisordersICD9 + attentionDisordersICD10

In [8]:
developmentalDisordersICD9 = diags['developmental_disorders']['icd9']
developmentalDisordersICD10 = diags['developmental_disorders']['icd10']
developmentalDisorders = developmentalDisordersICD9 + developmentalDisordersICD10

In [9]:
infancyChildhoodDisordersICD9 = diags['infancy_childhood_disorders']['icd9']
infancyChildhoodDisordersICD10 = diags['infancy_childhood_disorders']['icd10']
infancyChildhoodDisorders = infancyChildhoodDisordersICD9 + infancyChildhoodDisordersICD10

In [11]:
moodDisordersICD9 = diags['mood_disorders']['icd9']
moodDisordersICD10 = diags['mood_disorders']['icd10']
moodDisorders = moodDisordersICD9 + moodDisordersICD10

In [12]:
personalityDisordersICD9 = diags['personality_disorders']['icd9']
personalityDisordersICD10 = diags['personality_disorders']['icd10']
personalityDisorders = personalityDisordersICD9 + personalityDisordersICD10

In [13]:
schizophreniaPsychoticDisordersICD9 = diags['schizophrenia_psychotic_disorders']['icd9']
schizophreniaPsychoticDisordersICD10 = diags['schizophrenia_psychotic_disorders']['icd10']
schizophreniaPsychoticDisorders = schizophreniaPsychoticDisordersICD9 + schizophreniaPsychoticDisordersICD10

In [14]:
alcoholDisordersICD9 = diags['alcohol_disorders']['icd9']
alcoholDisordersICD10 = diags['alcohol_disorders']['icd10']
alcoholDisorders = alcoholDisordersICD9 + alcoholDisordersICD10

In [15]:
substanceDisordersICD9 = diags['substance_disorders']['icd9']
substanceDisordersICD10 = diags['substance_disorders']['icd10']
substanceDisorders = substanceDisordersICD9 + substanceDisordersICD10

In [16]:
suicideSelfHarmICD9 = diags['suicide_self_harm']['icd9']
suicideSelfHarmICD10 = diags['suicide_self_harm']['icd10']
suicideSelfHarm = suicideSelfHarmICD9 + suicideSelfHarmICD10

In [17]:
miscDisordersICD9 = diags['misc_disorders']['icd9']
miscDisordersICD10 = diags['misc_disorders']['icd10']
miscDisorders = miscDisordersICD9 + miscDisordersICD10

In [ ]:
#currently have medicaid data through 2012 so we will only use icd9

sql_query= get_counts_query(diags1)
data = pd.read_sql_query(sql_query, connection, index_col = 'bene_id').reset_index()

In [ ]:
print(diags1)